In [0]:
%python
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "d0471621-cb16-4841-ab1b-e9c0fc3370ef",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="azure-kv-scope",key="sp-secret"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/25235581-c428-4ef3-a446-dddcdfada476/oauth2/token"}
if not dbutils.fs.ls("/mnt/"):
  dbutils.fs.mount(
    source = "abfss://bhoojal@bhoojalstorageaccount.dfs.core.windows.net/",
    mount_point = "/mnt",
    extra_configs = configs)   
  

In [0]:
%python
%pip install geopandas
%pip install azure-cosmos

In [0]:
import math
import pandas as pd
import json
from shapely.geometry import Polygon
from shapely.geometry.polygon import orient
from pyspark.sql import functions as F

import geopandas as gpd
from azure.cosmos import exceptions, CosmosClient, PartitionKey
import uuid
from pyspark.sql import SparkSession

endpoint = "https://bhoojal-cosmos.documents.azure.com:443/"
key = '0dDLs6jiq9oopPFt9oUkxx3Fima5EVJ6yEbPYIVAZY1oHqhyYxZ5mXcghgqsMvAabsVsBfi5Samh5V8onFc1NQ=='
client = CosmosClient(endpoint, key)
database_name = 'bhoojal_outlets'
database = client.create_database_if_not_exists(id=database_name)

container_depth = database.create_container_if_not_exists(
    id="region_depth", 
    partition_key=PartitionKey(path="/city"),
    offer_throughput=400
)

fileName = dbutils.widgets.get('FileName')
gprData = spark.read.option("multiline", "true").json("/mnt/raw/" + fileName)

for row in gprData.collect():
  geoJson = {}
  depth = 0
  geoJson['id'] = str(uuid.uuid4())
  geoJson['scannedIn'] = row['scannedIn']
  geoJson['city'] = row['city']
  for observation in row['observations']:
    rowDepth = (float(observation['waveTravelTime']) * 0.3)/(math.sqrt(float(row['dielectric'])))
    depth = depth + rowDepth
    
  avgDepth = depth/len(row['observations'])
  geoJson['depth'] = avgDepth
    
  res = [(float(val['lng']), float(val['lat'])) for val in row['observations']]
  poly = orient(Polygon(res), sign=1.0)

  pg=gpd.GeoDataFrame(index=[0], geometry=[poly])
  jsonVal = json.loads(pg.to_json())
    
  for feature in jsonVal['features']:
    geoJson['boundary'] = feature['geometry']
  container_depth.create_item(body=geoJson)